This script aims to extract model sources in a clear and informative format.  The script first shows what all the kinetic and thermo sources are in a model. Then it goes through each reaction and species to show their source and what the assigned uncertainties are.  This can be used with any RMG-generated CHEMKIN file that is annotated.

In [1]:
from rmgpy.tools.uncertainty import Uncertainty, ThermoParameterUncertainty, KineticParameterUncertainty
from IPython.display import display
import copy
import numpy

In [2]:
chemFile = 'data/parseSource/chem_annotated.inp'
dictFile = 'data/parseSource/species_dictionary.txt'

In [3]:
uncertainty = Uncertainty(outputDirectory='testUncertainty')

In [4]:
uncertainty.loadModel(chemFile, dictFile)

In [5]:
uncertainty.loadDatabase()

In [6]:
uncertainty.extractSourcesFromModel()
uncertainty.compileAllSources()

In [7]:
print 'All Kinetic Sources'
for sourceType in uncertainty.allKineticSources.keys():
    if sourceType == 'Library':
        print '============'
        print 'Library kinetics'
        print ''
        print '\tReactions: ', uncertainty.allKineticSources['Library']
    elif sourceType == 'PDep':
        print '============'
        print 'PDep kinetics'
        print ''
        print '\tReactions: ', uncertainty.allKineticSources['PDep']
    elif sourceType == 'Rate Rules':
        print '============'
        print 'Rate rule kinetics'
        print ''
        for familyLabel, entries in uncertainty.allKineticSources['Rate Rules'].iteritems():
            print '\t', familyLabel
            for entry in entries:
                print '\t\t', entry
    elif sourceType == 'Training':
        print '============'
        print 'Training reaction kinetics'
        print ''
        for familyLabel, entries in uncertainty.allKineticSources['Training'].iteritems():
            print '\t', familyLabel
            for entry in entries:
                print '\t\t', entry
    else:
        print sourceType
        raise Exception('Kinetics source must be Library, PDep, Rate Rules, or Training')

All Kinetic Sources
Rate rule kinetics

	H_Abstraction
		OH_rad_H;N_atom_quartet
		O/H/OneDe;C_rad/H2/Ct
		C/H3/Cd;C_methyl
		C/H3/Cd;C_rad/H2/Cs
		C/H3/CS;C_rad/H2/CS
		C/H3/Cd\Cs_Cd\H2;C_rad/H2/Cs\H\Cs\Cs|O
		H2O2;CO_rad/Cs
		Cd/H/Cd;Cd_Cdd_rad/H
		C/H3/Cd;C_rad/H2/Cd
		Cd/H/NonDeC;Cd_Cdd_rad/H
		Cd_Cdd/H2;Cd_rad/NonDeS
		C/H3/Cd\Cs_Cd\H2;O_rad/Cs\H2\Cs|H|Cs2
		C/H3/Cd\H_Cd\H2;C_rad/H2/Cs\H\Cs\Cs|O
		Cd_Cdd/H2;C_rad/H/CtCt
		NH2_rad_H;O_pri_rad
		H2;N_atom_quartet
		H2O2;C_rad/H2/Cs\H\Cs\Cs|O
		C/H3/Cs;Cd_Cd\H2_pri_rad
		Cd_Cdd/H2;C_rad/CSCsS
		CO_pri;O2b
		CO_pri;O_atom_triplet
		C/H3/Cd;O_rad/NonDeO
		CO_pri;CH2_triplet
		C/H3/Ct;C_rad/H2/Cb
		C/H2/CbCs;Cd_Cdd_rad/H
		C/H3/Cs\H3;C_rad/H2/Cs\H\Cs\Cs|O
		Cd_Cdd/H2;C_rad/Cs2S
		CO_pri;C_rad/H2/Cs
		C/H2/NonDeC;O_rad/NonDeO
		CO_pri;C_rad/H/NonDeC
		Cd_Cdd/H2;C_rad/CdCdCs
		O/H/OneDeC;H_rad
		CO_pri;C_rad/Cs3
		C/H3/Cb;O_rad/NonDeO
		CO_pri;Cd_pri_rad
		C_methane;Cd_Cdd_rad/H
		C_methane;O_rad/NonDeO
		CO_pri;CO_rad/NonDe
		CO_pri;O_pr

In [8]:
print 'All Thermo Sources'
for sourceType in uncertainty.allThermoSources.keys():
    if sourceType == 'Library':
        print '============'
        print 'Library thermo'
        print ''
        print '\tSpecies: ', uncertainty.allThermoSources['Library']
    elif sourceType == 'QM':
        print '============'
        print 'QM thermo'
        print ''
        print '\tSpecies: ', uncertainty.allThermoSources['QM']
    elif sourceType == 'GAV':
        print '============'
        print 'Group additivity thermo'
        print ''
        for groupType, entries in uncertainty.allThermoSources['GAV'].iteritems():
            print '\t', groupType
            for entry in entries:
                print '\t\t', entry
    else:
        raise Exception('Thermo source must be GAV, QM, or Library')

All Thermo Sources
Group additivity thermo

	radical
		CsJOH
		HCdsJO
		CH3
		CCJ
		H3COJ
		Cs_P
		C=COJ
		Cds_P
		HOOJ
		Acetyl
	other
		ketene
		R
	group
		Ct-CtCs
		Od-Cd
		Cs-(Cds-Od)HHH
		Cds-(Cdd-Od)HH
		Cds-CdsHH
		Cs-CsHHH
		Cds-CdsOsH
		Cs-OsHHH
		Cds-OdHH
		Cds-OdCsH
		Cdd-OdOd
		Ct-CtH
		Cs-CsCsHH
		Ct-CtOs
		Os-CsH
		Os-OsH
		Os-CsCs
		Cdd-CdsOd
		Os-CtH
QM thermo

	Species:  []
Library thermo

	Species:  [32, 1, 2, 3, 5, 6, 7, 8, 10, 14, 16, 19, 0, 31]


In [9]:
# Assign all the uncertainties using the Uncertainty() class function
# ThermoParameterUncertainty and KineticParameterUncertainty classes may be customized and passed into this function
# if non-default constants for constructing the uncertainties are desired
uncertainty.assignParameterUncertainties()

In [10]:
T = 623 # temperature in Kelvin for which to evaluate kinetics
P = 1e5  # Pa 

In [11]:
gParamEngine = ThermoParameterUncertainty()
kParamEngine = KineticParameterUncertainty()

In [12]:
for rxn, source in uncertainty.reactionSourcesDict.iteritems():
    print '======'
    print rxn
    display(rxn)
    if 'Library' in source:
        print 'Library reaction'
        print source['Library']
    elif 'PDep' in source:
        print 'PDep reaction'
        print source['PDep']
    elif 'Rate Rules' in source:
        print 'Rate rule estimate'
        family = source['Rate Rules'][0]
        sourceDict = source['Rate Rules'][1]
        originalTemplate = sourceDict['template']
        print '\tFamily = ', family
        print '\tOriginal Template = ', [group.label for group in originalTemplate]
        print '\tExact = ', sourceDict['exact']
        rules = sourceDict['rules']
        training = sourceDict['training']
        if rules:
            print '\tRate rule sources:'
            for ruleEntry, weight in rules:
                print '\t\t', ruleEntry, '=', weight
        if training:
            print '\tTraining sources:'
            for ruleEntry, trainingEntry, weight in training:
                print '\t\t', ruleEntry , 'mapped to', trainingEntry , '=', weight
    elif 'Training' in source:
        print 'Training reaction'
        family = source['Training'][0]
        training = source['Training'][1]
        print '\t Family = ', family
        print '\t\t', training



    print ''
    print 'Rate coefficient at {} K = {:.2e}'.format(T, rxn.kinetics.getRateCoefficient(T,P))

    # Uncomment the following lines if you want to verify that the parsing has been performed correctly by
    # checking the values for both the original and reconstructed kinetics

#     print '---------'
#     print 'Original kinetics:'
#     print rxn.kinetics
#     print ''
#     print 'Reconstructed kinetics from parsing:'
#     reconstructedKinetics=uncertainty.database.kinetics.reconstructKineticsFromSource(rxn,source,fixBarrierHeight=True)
#     print reconstructedKinetics

    rxnIndex = uncertainty.reactionList.index(rxn)
    print 'Uncertainty dln(k) = ', uncertainty.kineticInputUncertainties[rxnIndex]
    
#     # Test that the partial uncertainty calculation is working
#     dlnk = 0.0
#     if 'Rate Rules' in source:
#         family = source['Rate Rules'][0]
#         sourceDict = source['Rate Rules'][1]
#         rules = sourceDict['rules']
#         training = sourceDict['training']
#         for ruleEntry, weight in rules:
#             dlnk += kParamEngine.getPartialUncertaintyValue(source, 'Rate Rules', corrParam=ruleEntry, corrFamily=family)
#         for ruleEntry, trainingEntry, weight in training:
#             dlnk += kParamEngine.getPartialUncertaintyValue(source, 'Rate Rules', corrParam=ruleEntry, corrFamily=family)
#         dlnk += kParamEngine.getPartialUncertaintyValue(source, 'Estimation')
#     elif 'PDep' in source:
#         dlnk += kParamEngine.getPartialUncertaintyValue(source, 'PDep', source['PDep'])
#     elif 'Library' in source:
#         dlnk += kParamEngine.getPartialUncertaintyValue(source, 'Library', source['Library'])
#     elif 'Training' in source:
#         dlnk += kParamEngine.getPartialUncertaintyValue(source, 'Training', source['Training'])
#     print 'Uncertainty dlnk calculated using sum of partial values = ', dlnk
    
    

CH2OH(18) + C(29) <=> CH2O(15) + CH(9)


Rate rule estimate
	Family =  Disproportionation
	Original Template =  ['C_triplet', 'O_Csrad']
	Exact =  False
	Rate rule sources:
		O_rad/NonDeO;O_Csrad = 0.0138888888889
		C_rad/Cs3;O_Csrad = 0.0138888888889
		O_atom_triplet;O_Csrad = 0.166666666667
		C_rad/H2/Cd;O_Csrad = 0.00462962962963
		CH2_triplet;O_Csrad = 0.166666666667
		C_rad/H/NonDeC;O_Csrad = 0.0138888888889
		H_rad;O_Csrad = 0.0555555555556
		C_rad/H2/Cs;O_Csrad = 0.00462962962963
		Cd_pri_rad;O_Csrad = 0.0555555555556
		C_rad/H2/O;O_Csrad = 0.00462962962963
		CO_pri_rad;O_Csrad = 0.0555555555556
		O_pri_rad;O_Csrad = 0.0277777777778
		O2b;O_Csrad = 0.333333333333
		O_rad/NonDeC;O_Csrad = 0.0138888888889
		C_methyl;O_Csrad = 0.0138888888889
	Training sources:
		Ct_rad/Ct;O_Csrad mapped to C2H + CH3O <=> C2H2 + CH2O = 0.0555555555556

Rate coefficient at 623 K = 2.36e+07
Uncertainty dln(k) =  5.80657122482
H2(3) + CH2(S)(13) <=> CH4(16)


PDep reaction
380

Rate coefficient at 623 K = 1.03e+01
Uncertainty dln(k) =  2.0
CH2O(15) + CH(9) <=> HCO(12) + CH2(S)(13)


Rate rule estimate
	Family =  H_Abstraction
	Original Template =  ['CO_pri', 'CH_doublet']
	Exact =  False
	Rate rule sources:
		CO_pri;C_rad/Cs3 = 0.025
		CO_pri;Cd_pri_rad = 0.1
		CO_pri;CH2_triplet = 0.25
		CO_pri;O2b = 0.1
		CO_pri;O_atom_triplet = 0.25
		CO_pri;O_pri_rad = 0.05
		CO_pri;O_rad/NonDeC = 0.0166666666667
		CO_pri;C_rad/H/NonDeC = 0.025
		CO_pri;O_rad/NonDeO = 0.0166666666667
		CO_pri;CO_rad/NonDe = 0.1
		CO_pri;C_rad/H2/Cs = 0.0125
		CO_pri;C_rad/H2/Cd\Cs_Cd\H2 = 0.0125
		CO_pri;C_methyl = 0.025
	Training sources:
		CO_pri;O_rad/OneDeN mapped to CH2O + NO2 <=> CHO_p1 + HONO_p = 0.0166666666667

Rate coefficient at 623 K = 2.41e+03
Uncertainty dln(k) =  5.61631940669
CH3O(19) + CH3(14) <=> CH4(16) + CH2O(15)


Rate rule estimate
	Family =  Disproportionation
	Original Template =  ['C_methyl', 'Cmethyl_Orad']
	Exact =  False
	Rate rule sources:
		C_methyl;Cmethyl_Csrad = 0.5
		C_methyl;Cmethyl_Nrad = 0.5

Rate coefficient at 623 K = 4.95e+06
Uncertainty dln(k) =  3.16992439152
CH2(S)(13) + CH3(14) <=> CH4(16) + CH(9)


...

Library reaction
166

Rate coefficient at 623 K = 1.10e+07
Uncertainty dln(k) =  0.5
HO2(6) + CH(9) <=> C(29) + H2O2(8)


Rate rule estimate
	Family =  H_Abstraction
	Original Template =  ['C_doublet_H', 'O_rad/NonDeO']
	Exact =  False
	Rate rule sources:
		C/H3/Cb;O_rad/NonDeO = 0.00416666666667
		N3s/H2/NonDeN;O_rad/NonDeO = 0.0208333333333
		C/H2/Cd\H_Cd\H2/Cs\H3;OOC = 0.316666666667
		H2O2;OOC = 0.354166666667
		H2O2;O_rad/NonDeO = 0.00694444444444
		C/H3/Cs;O_rad/NonDeO = 0.00833333333333
		C/H2/NonDeC;O_rad/NonDeO = 0.00555555555556
		C/H/Cs2Cb;O_rad/NonDeO = 0.00833333333333
		C/H/Cs3;O_rad/NonDeO = 0.00833333333333
		Orad_O_H;O_rad/NonDeO = 0.0833333333333
		O/H/NonDeN;O_rad/NonDeO = 0.0138888888889
		C/H2/OneDeC;O_rad/NonDeO = 0.00277777777778
		CO/H/NonDe;O_rad/NonDeO = 0.0416666666667
		C_methane;O_rad/NonDeO = 0.0166666666667
		N3s/H2/NonDeO;O_rad/NonDeO = 0.0208333333333
		C/H3/Cd;O_rad/NonDeO = 0.00416666666667
		N3s/H2/OneDeN;O_rad/NonDeO = 0.0416666666667
		CO_pri;O_rad/NonDeO = 0.0416666666667

Rate coefficient at 623 K = 1.49e+02
Uncertainty dln(k) =  5.97563760333
CH2CHO(31) + HO2(6) <=

Rate rule estimate
	Family =  Disproportionation
	Original Template =  ['O_rad/NonDeO', 'Cdpri_Orad']
	Exact =  False
	Rate rule sources:
		O_rad/NonDeO;O_Csrad = 0.25
		O_rad/NonDeO;N3s/H2_s_Nrad = 0.25
		O_rad/NonDeO;N3s/H2_s_Orad = 0.25
		O_rad/NonDeO;O_Nrad = 0.25

Rate coefficient at 623 K = 4.78e+06
Uncertainty dln(k) =  3.94639501518
C2H5(27) + CH3(14) <=> C3H8(33)


Library reaction
198

Rate coefficient at 623 K = 9.06e+06
Uncertainty dln(k) =  0.5
C2H5(27) + HCCOH(30) <=> ethane(1) + HCCO(23)


Rate rule estimate
	Family =  H_Abstraction
	Original Template =  ['O/H/OneDeC', 'C_rad/H2/Cs\\H3']
	Exact =  False
	Rate rule sources:
		H2O2;C_rad/H2/Cs = 0.00892857142857
		ROOH_sec;C_rad/H2/Cs = 0.00892857142857
		ROOH_pri;C_rad/H2/Cs = 0.00892857142857
		H2O2;C_rad/H2/Cs\H\Cs\Cs|O = 0.0803571428571
		H2O2;C_rad/H2/Cs\Cs2\O = 0.0803571428571
		O/H/NonDeC;C_rad/H2/Cs = 0.0714285714286
		H2O2;C_rad/H2/Cs\H2\Cs|Cs#O = 0.0803571428571
		H2O2;C_rad/H2/Cs\H2\Cs|Cs|O = 0.0803571428571
		O/H/OneDe;C_rad/H2/Ct = 0.5
	Training sources:
		H2O2;C_rad/H2/Cs\H\Cs|Cs\O mapped to H2O2 + C4H9O-8 <=> HO2 + C4H10O-8 = 0.0803571428571

Rate coefficient at 623 K = 7.57e+01
Uncertainty dln(k) =  5.14487439805


In [13]:
for species, source in uncertainty.speciesSourcesDict.iteritems():
    print '=========='
    print species
    display(species)
    if 'Library' in source:
        print 'Thermo Library: ', source['Library']
    if 'QM' in source:
        print 'QM: ', source['QM']
    if 'GAV' in source:
        print 'Group additivity:'
        for groupType, groupList in source['GAV'].iteritems():
            print '\t', groupType
            for group, weight in groupList:
                print '\t\t', group, '=', weight
                
    
                
    spcIndex = uncertainty.speciesList.index(species)    
    print ''
    print 'Uncertainty dG = ', uncertainty.thermoInputUncertainties[spcIndex], ' kcal/mol'
    
    
    # Test that the partial uncertainty calculation is working
    dG = 0.0
    if 'Library' in source:
        dG += gParamEngine.getPartialUncertaintyValue(source, 'Library', corrParam=source['Library'])
    if 'QM' in source:
        dG += gParamEngine.getPartialUncertaintyValue(source, 'QM',corrParam=source['QM'])
    if 'GAV' in source:
        for groupType, groupList in source['GAV'].iteritems():
            for group, weight in groupList:
                dG += gParamEngine.getPartialUncertaintyValue(source, 'GAV', group, groupType)
        dG += gParamEngine.getPartialUncertaintyValue(source, 'Estimation')
    print 'Uncertainty dG calculated using sum of partial values = ', dG, ' kcal/mol'

CH2CO(25)


Group additivity:
	other
		ketene = 1
		R = 2
	group
		Od-Cd = 1
		Cds-(Cdd-Od)HH = 1
		Cdd-CdsOd = 1

Uncertainty dG =  2.1  kcal/mol
Uncertainty dG calculated using sum of partial values =  2.1  kcal/mol
CH3(14)


Thermo Library:  19
Group additivity:
	radical
		CH3 = 1

Uncertainty dG =  3.1  kcal/mol
Uncertainty dG calculated using sum of partial values =  3.1  kcal/mol
HCCOH(30)


Group additivity:
	other
		R = 3
	group
		Ct-CtH = 1
		Ct-CtOs = 1
		Os-CtH = 1

Uncertainty dG =  2.1  kcal/mol
Uncertainty dG calculated using sum of partial values =  2.1  kcal/mol
O(2)


Thermo Library:  5

Uncertainty dG =  1.5  kcal/mol
Uncertainty dG calculated using sum of partial values =  1.5  kcal/mol
CH2O(15)


...

Group additivity:
	radical
		HCdsJO = 1
	other
		R = 2
	group
		Od-Cd = 1
		Cds-OdHH = 1

Uncertainty dG =  2.0  kcal/mol
Uncertainty dG calculated using sum of partial values =  2.0  kcal/mol
C2H4(26)


Group additivity:
	other
		R = 2
	group
		Cds-CdsHH = 2

Uncertainty dG =  1.9  kcal/mol
Uncertainty dG calculated using sum of partial values =  1.9  kcal/mol
He


Thermo Library:  2

Uncertainty dG =  1.5  kcal/mol
Uncertainty dG calculated using sum of partial values =  1.5  kcal/mol
CH2(S)(13)


Thermo Library:  16

Uncertainty dG =  1.5  kcal/mol
Uncertainty dG calculated using sum of partial values =  1.5  kcal/mol
N2


Thermo Library:  0

Uncertainty dG =  1.5  kcal/mol
Uncertainty dG calculated using sum of partial values =  1.5  kcal/mol


In [14]:
# Assign correlated parameter uncertainties 
uncertainty.assignParameterUncertainties(correlated=True)

In [15]:
# See the thermo correlated parameter partial uncertainties
uncertainty.thermoInputUncertainties

[{'Library N2': 1.5},
 {'Library Ar': 1.5},
 {'Library He': 1.5},
 {'Library Ne': 1.5},
 {'Estimation ethane(1)': 1.5,
  'Group(group) Cs-CsHHH': 0.2,
  'Group(other) R': 0.2},
 {'Library O(2)': 1.5},
 {'Library H2(3)': 1.5},
 {'Library H(4)': 1.5},
 {'Library OH(5)': 1.5},
 {'Estimation HO2(6)': 1.5,
  'Group(group) Os-OsH': 0.2,
  'Group(other) R': 0.2,
  'Group(radical) HOOJ': 0.1},
 {'Library O2(7)': 1.5},
 {'Estimation H2O2(8)': 1.5,
  'Group(group) Os-OsH': 0.2,
  'Group(other) R': 0.2},
 {'Estimation CH(9)': 1.5,
  'Group(radical) Cs_P': 0.1,
  'Library CH2(S)(13)': 1.5},
 {'Estimation CO(10)': 1.5,
  'Group(group) Ct-CtCs': 0.1,
  'Group(group) Os-CsCs': 0.1,
  'Group(other) R': 0.2},
 {'Library CH2(11)': 1.5},
 {'Estimation HCO(12)': 1.5,
  'Group(group) Cds-OdHH': 0.1,
  'Group(group) Od-Cd': 0.1,
  'Group(other) R': 0.2,
  'Group(radical) HCdsJO': 0.1},
 {'Library CH2(S)(13)': 1.5},
 {'Estimation CH3(14)': 1.5,
  'Group(radical) CH3': 0.1,
  'Library CH4(16)': 1.5},
 {'Estim

In [16]:
# See the kinetics correlated parameter partial uncertainties
uncertainty.kineticInputUncertainties

[{'Library O(2)+H2(3)=H(4)+OH(5)': 0.5},
 {'Library O(2)+HO2(6)=OH(5)+O2(7)': 0.5},
 {'Library O(2)+H2O2(8)=OH(5)+HO2(6)': 0.5},
 {'Library O(2)+CH(9)=H(4)+CO(10)': 0.5},
 {'Library O(2)+CH2(11)=H(4)+HCO(12)': 0.5},
 {'Library O(2)+CH2(S)(13)=H2(3)+CO(10)': 0.5},
 {'Library O(2)+CH2(S)(13)=H(4)+HCO(12)': 0.5},
 {'Library O(2)+CH3(14)=H(4)+CH2O(15)': 0.5},
 {'Library CH4(16)+O(2)=OH(5)+CH3(14)': 0.5},
 {'Library O(2)+HCO(12)=OH(5)+CO(10)': 0.5},
 {'Library O(2)+HCO(12)=CO2(17)+H(4)': 0.5},
 {'Library O(2)+CH2O(15)=OH(5)+HCO(12)': 0.5},
 {'Library CH2OH(18)+O(2)=OH(5)+CH2O(15)': 0.5},
 {'Library CH3O(19)+O(2)=OH(5)+CH2O(15)': 0.5},
 {'Library CH3OH(20)+O(2)=CH2OH(18)+OH(5)': 0.5},
 {'Library CH3OH(20)+O(2)=CH3O(19)+OH(5)': 0.5},
 {'Library C2H(21)+O(2)=CH(9)+CO(10)': 0.5},
 {'Library C2H2(22)+O(2)=HCCO(23)+H(4)': 0.5},
 {'Library C2H2(22)+O(2)=C2H(21)+OH(5)': 0.5},
 {'Library C2H2(22)+O(2)=CO(10)+CH2(11)': 0.5},
 {'Library C2H3(24)+O(2)=CH2CO(25)+H(4)': 0.5},
 {'Library C2H4(26)+O(2)=HCO